# D1 - Walvisualisatie
Noor Koevoet, Glenn de Graaff, Anne Beks, Job Kolhorn

D1

## Introductie
Persvrijheid is een principe dat stelt dat de pers vrij is om te schrijven, zeggen en publiceren wat zij willen. De overheid is in beginsel verboden om censuur toe te passen en mag niet vooraf controleren wat wel of niet gezegd mag worden. In Nederland is persvrijheid, samen met het recht op meningsuiting, vastgelegd in artikel 7 van de Grondwet. Ook in andere landen en binnen Europees verband zijn er wetten die persvrijheid en vrijheid van meningsuiting beschermen.

Om te verzekeren dat journalisten hun werk veilig kunnen doen, hebben zij een beschermde status van het Europese Hof voor de Rechten van de Mens (EHRM). Journalisten brengen informatie naar buiten over algemeen belangrijke zaken en zijn daarom van cruciaal belang voor de democratie. Hoewel er vrijheid is om te publiceren wat men wil, betekent dit niet dat dit onbestraft kan gebeuren. Wanneer publicaties discriminerend zijn of aanzetten tot opruiing, kan iemand aangeklaagd en veroordeeld worden. Dit gebeurt echter pas achteraf; vóór publicatie is censuur niet toegestaan.

In veel landen is persvrijheid echter niet gegarandeerd. In landen met een autocratisch regime is de vrijheid van meningsuiting vaak beperkt of volledig afwezig. Ook in landen waar conflicten gaande zijn, zijn er mogelijk meerdere machten aan de leiding, of lijkt er niemand aan de macht te zijn. In zulke situaties is het moeilijk om te waarborgen dat de wet wordt nageleefd.

Deze data-analyse zal onderzoeken of de mate van persvrijheid wordt beïnvloed door conflicten of de mate van democratie. Door gebruik te maken van data van Verslaggevers Without Borders (RSF), The Armed Conflict Location & Event Data Project (ACLED) en Our World in Data, zullen we scores en ranglijsten analyseren rondom persvrijheid, democratie en conflicten wereldwijd. Het perspectief dat conflicten de persvrijheid verminderen en het perspectief dat autocratische bestuursvormen de persvrijheid verminderen, worden hier onderzocht.

## Dataset and Preprocessing

tekst

## Eerste perspective

tekst

In [53]:
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
import plotly.express as px

import ipywidgets as widgets
from ipywidgets import interact

import numpy as np

from IPython.display import display

init_notebook_mode(connected=True)

### Eerste argument van eeste perspectief

Om meerdere landen te selecteren ctrl indrukken

In [54]:
data_pers_ranking_19 = pd.read_csv('./data/2019.csv', sep=';')
data_pers_ranking_23 = pd.read_csv('./data/2023.csv', sep=';')

data_conflict = pd.read_excel('./data/01-2024-conflic-index.xlsx')

data_pers_ranking_19.rename(columns={'EN_country': 'country', 'Rank N': 'Rank_2019'}, inplace=True)
data_pers_ranking_23.rename(columns={'Country_EN': 'country', 'Rank': 'Rank_2023'}, inplace=True)
data_conflict.rename(columns={'Country': 'country', 'Index Ranking 2023': 'Conflict_Rank_2023', 'Change in ranking since 2019': 'Change_Conflict'}, inplace=True)

data_pers_combined = pd.merge(data_pers_ranking_19[['country', 'Rank_2019']], 
                              data_pers_ranking_23[['country', 'Rank_2023']], 
                              on='country', 
                              how='inner')


data_conflict['Conflict_Rank_2019'] = data_conflict['Conflict_Rank_2023'] + data_conflict['Change_Conflict']


def filter_countries(text, dataset):
    if dataset == 'Conflict':
        return [country for country in data_conflict['country'] if text.lower() in country.lower()]
    else:
        return [country for country in data_pers_combined['country'] if text.lower() in country.lower()]


def plot_countries_ranking(countries):
    if not countries:
        print("Selecteer minstens een land")
        return
    
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=('Conflict Index', 'Press Freedom Index'))
    
    for country in countries:

        selected_data_conflict = data_conflict[data_conflict['country'] == country]
        
        if len(selected_data_conflict) > 0:
            ranking_2019 = selected_data_conflict['Conflict_Rank_2019'].values[0]
            ranking_2023 = selected_data_conflict['Conflict_Rank_2023'].values[0]
            change = ranking_2023 - ranking_2019
            
            if change == 0:
                color = 'blue'
            elif change > 0:
                color = 'green'
            else:
                color = 'red' 
            
            fig.add_trace(go.Scatter(
                x = ['2019', '2023'],
                y = [ranking_2019, ranking_2023],
                mode = 'lines+markers',
                marker = dict(color=color),
                line = dict(color=color),
                name = f'{country} Conflict ({ranking_2019} -> {ranking_2023})'
            ), row=1, col=1)
        
        selected_data_pers = data_pers_combined[data_pers_combined['country'] == country]
        
        if len(selected_data_pers) > 0:
            ranking_2019 = selected_data_pers['Rank_2019'].values[0]
            ranking_2023 = selected_data_pers['Rank_2023'].values[0]
            change = ranking_2023 - ranking_2019
            
            if change == 0:
                color = 'blue'
            elif change > 0:
                color = 'red'
            else:
                color = 'green' 
            
            fig.add_trace(go.Scatter(
                x = ['2019', '2023'],
                y = [ranking_2019, ranking_2023],
                mode = 'lines+markers',
                marker = dict(color=color),
                line = dict(color=color),
                name = f'{country} Press Freedom ({ranking_2019} -> {ranking_2023})'
            ), row=2, col=1)
        
    if len(countries) == 1:
        title = f'Verandering in Ranking voor {countries[0]}'
    elif len(countries) == 2:
        title = f'Verandering in Ranking voor {" en ".join(countries)}'
    elif len(countries) > 5:
        title = f'Verandering in Ranking voor {len(countries)} landen'
    else:
        title = f'Verandering in Ranking voor {", ".join(countries)}'
        
    fig.update_layout(
        height=800,
        title_text=title,
        showlegend=True,
    )
    
    fig.update_xaxes(title_text='Jaar', row=1, col=1)
    fig.update_xaxes(title_text='Jaar', row=2, col=1)
    fig.update_yaxes(title_text='Conflict Index Ranking', row=1, col=1)
    fig.update_yaxes(title_text='Press Freedom Index Ranking', row=2, col=1)
    
    fig.show()

dataset_dropdown = widgets.Dropdown(
    options=['Conflict', 'Press Freedom'],
    value='Conflict',
    description='Dataset:'
)

country_dropdown = widgets.SelectMultiple(
    options = data_conflict['country'].unique().tolist() if dataset_dropdown.value == 'Conflict' else data_pers_combined['country'].unique().tolist(),
    description = 'Landen:',
    disabled = False,
    layout = widgets.Layout(height='200px', width='400px'),
)

search_text = widgets.Text(
    value = '',
    placeholder = 'Zoek landen',
    description = 'Zoeken:',
    disabled = False
)

def handle_search(change):
    search_results = filter_countries(change.new, dataset_dropdown.value)
    
    selected_countries = list(country_dropdown.value)
    country_dropdown.options = search_results
    
    country_dropdown.value = [country for country in selected_countries if country in search_results]

def update_country_dropdown(change):
    search_results = filter_countries(search_text.value, change.new)
    country_dropdown.options = search_results
    country_dropdown.value = []

search_text.observe(handle_search, names='value')
dataset_dropdown.observe(update_country_dropdown, names='value')

display(dataset_dropdown)
display(search_text)

interact(plot_countries_ranking, countries=country_dropdown)

Dropdown(description='Dataset:', options=('Conflict', 'Press Freedom'), value='Conflict')

Text(value='', description='Zoeken:', placeholder='Zoek landen')

interactive(children=(SelectMultiple(description='Landen:', layout=Layout(height='200px', width='400px'), opti…

<function __main__.plot_countries_ranking(countries)>

tekst

In [55]:
df = pd.DataFrame(data_conflict)

data_conflict['Conflict_Rank_2019'] = data_conflict['Conflict_Rank_2023'] + data_conflict['Change_Conflict']

conditions = [
    (df['Change_Conflict'] > 0),
    (df['Change_Conflict'] < 0),
    (df['Change_Conflict'] == 0)
]
choices = ['Verslechterd', 'Verbeterd', 'Gelijk gebleven']
df['Status'] = np.select(conditions, choices, default='Niet gedefinieerd')

status_counts = df['Status'].value_counts()

fig_status = go.Figure(data=[go.Pie(
    labels=status_counts.index,
    values=status_counts.values,
    marker=dict(colors=['green', 'red', 'blue'])
)])

fig_status.update_layout(
    title='Verandering in Conflict Index Rankings sinds 2019 per 2023',
    legend_title='Status'
)

fig_status.show()

tekst

In [56]:
data_pers_combined['Change'] = data_pers_combined['Rank_2023'] - data_pers_combined['Rank_2019']

conditions = [
    (data_pers_combined['Change'] > 0),
    (data_pers_combined['Change'] < 0),
    (data_pers_combined['Change'] == 0)
]
choices = ['Verbeterd', 'Verslechterd', 'Gelijk gebleven']
data_pers_combined['Status'] = np.select(conditions, choices, default='Niet gedefinieerd')

status_counts = data_pers_combined['Status'].value_counts()

fig_status = go.Figure(data=[go.Pie(
    labels=status_counts.index,
    values=status_counts.values,
    marker=dict(colors=['green', 'red', 'blue'])
)])

fig_status.update_layout(
    title='Verandering in Persvrijheidsindex Rankings sinds 2019 per 2023',
    legend_title='Status'
)

fig_status.show()

### Tweede argument: eerste perspectief

tekst

In [57]:
conflict_data = pd.read_excel('./data/01-2024-conflic-index.xlsx')
data_conflict_ranking = conflict_data[['country', 'Index Ranking 2023']]

data_pers = pd.read_csv('./data/2023.csv', sep=';')
data_pers.rename(columns = {'Country_EN':'country'}, inplace=True)

data_pers_ranking = data_pers[['country', 'Rank']]

data_both = pd.merge(data_pers_ranking, data_conflict_ranking, on='country', how='inner')

def determine_color(rank):
    if rank <= 10:
        return 'red'
    elif rank <= 30:
        return 'orange'
    elif rank <= 50:
        return 'yellow'
    else:
        return 'blue'

df = pd.DataFrame(data_both)
df['Color'] = df['Index Ranking 2023'].apply(determine_color)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df['Index Ranking 2023'],
    y=df['Rank'],
    mode='markers',
    text=df['country'],
    marker=dict(
        color=df['Color'],
        size=10
    ),
    name = 'Land',
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=[None], y=[None],
    mode='markers',
    marker=dict(color='red', size=10),
    legendgroup='group1',
    showlegend=True,
    name='Extreem (Top 10)'
))
fig.add_trace(go.Scatter(
    x=[None], y=[None],
    mode='markers',
    marker=dict(color='orange', size=10),
    legendgroup='group2',
    showlegend=True,
    name='Hoog (11-30)'
))
fig.add_trace(go.Scatter(
    x=[None], y=[None],
    mode='markers',
    marker=dict(color='yellow', size=10),
    legendgroup='group3',
    showlegend=True,
    name='Turbulent (31-50)'
))
fig.add_trace(go.Scatter(
    x=[None], y=[None],
    mode='markers',
    marker=dict(color='blue', size=10),
    legendgroup='group4',
    showlegend=True,
    name='Laag/inactief (>50)'
))

fig.add_trace(go.Scatter(
    x=[min(df['Index Ranking 2023']), max(df['Index Ranking 2023'])],
    y=[max(df['Rank']), min(df['Rank'])],
    mode='lines',
    line=dict(color='black'),
    name='Trend lijn'
))

fig.update_layout(
    title = 'Press Freedom Index Ranking vs Conflict Index Ranking (2023)',
    xaxis_title = 'Conflict Index Ranking (Index Ranking 2023)',
    yaxis_title = 'Press Freedom Index Ranking (Rank)',
    showlegend = True,
    height = 700
)

fig.show()

correlation = df['Index Ranking 2023'].corr(df['Rank'])

print(f"Correlatie tussen de conflict index ranking en de persvrijheid index ranking: {correlation}")

Correlatie tussen de conflict index ranking en de persvrijheid index ranking: -0.42573893484944786


tekst

In [58]:
df['Color'] = df['Rank'].apply(determine_color)

color_counts = df['Color'].value_counts()

color_labels = {
    'red': 'Extreem',
    'orange': 'Hoog',
    'yellow': 'Turbulent',
    'blue': 'laag/inactief'
}


labels = [color_labels[color] for color in color_counts.index]


fig = go.Figure(data=[go.Pie(
    labels = labels,
    values = color_counts.values,
    marker = dict(colors = color_counts.index)
)])

fig.update_layout(
    title = 'Aantal landen met bepaalde conflict status (index level) ',
    legend_title = 'Index level'
)

fig.show()

Tekst

## Tweede perspectief

### Data voor visualisatie 1 & 2

Persvrijheid data voorbereiden

In [59]:
# create a dataframe for the pressfreedom data of every year, renamed a few columns from the latest three datasets so they are the same as the rest
df_24 = pd.read_csv('./data/persvrijheid/2024.csv', sep = ';').rename(columns={"Score": "Score N", "Rank": "Rank N", "Country_FR": "FR_country", "Country_EN": "EN_country", "Country_ES": "ES_country", "Country_FA": "FA_country", "Country_AR": "AR_country"})

df_23 = pd.read_csv('./data/persvrijheid/2023.csv', sep = ';').rename(columns={"Score": "Score N", "Rank": "Rank N", "Country_FR": "FR_country", "Country_EN": "EN_country", "Country_ES": "ES_country", "Country_FA": "FA_country", "Country_AR": "AR_country"})

df_22 = pd.read_csv('./data/persvrijheid/2022.csv', sep = ';').rename(columns={"Score": "Score N", "Rank": "Rank N", "Country_FR": "FR_country", "Country_EN": "EN_country", "Country_ES": "ES_country", "Country_FA": "FA_country", "Country_AR": "AR_country"})

df_21 = pd.read_csv('./data/persvrijheid/2021.csv', sep = ';')
df_20 = pd.read_csv('./data/persvrijheid/2020.csv', sep = ';')
df_19 = pd.read_csv('./data/persvrijheid/2019.csv', sep = ';')

In [60]:
# combineer de data voor elk land vanaf het jaar 2019

df_combined_19 = pd.concat([df_24, df_23, df_22, df_21, df_20, df_19])
df_grouped_19 = df_combined_19.groupby(by='ISO')

Democratie data (2023) voorbereiden:

In [61]:
# maak de data klaar voor de persvrijheid en democratie scores van 2023

data_23 = pd.read_csv('./data/persvrijheid/2023.csv', sep = ';')
data_dem = pd.read_csv('./data/democracy.csv')
data_dem_23 = data_dem[data_dem["Year"]==2023].sort_values(by="Democracy score", ascending=False).reset_index().rename(columns={'Entity':'Country_EN'})

In [62]:
# geef de landen een ranking gebaseerd om de democratie score

score = data_dem_23.iloc[0]["Democracy score"]
rank = 1
ranking = []
for country in data_dem_23.index:
    count = country + 1
    if len(ranking)==0:
        ranking.append(1)
    else: 
        if data_dem_23.iloc[country]["Democracy score"] == data_dem_23.iloc[country-1]["Democracy score"]:
            ranking.append(ranking[-1])
        elif data_dem_23.iloc[country]["Democracy score"] < data_dem_23.iloc[country-1]["Democracy score"]:
            ranking.append(count)
            
data_dem_23["ranking"] = ranking

In [63]:
#join de democratie en persvrijheid data tot één dataframe
data_joined = pd.merge(data_dem_23, data_23, on='Country_EN', how='inner')

### Data voor visualisatie 3

Overige democratie data voorbereiden (vanaf 2019)

In [64]:
# maak ook de democratie data klaar voor de andere jaartallen vanaf 2019 (voor 2024 is er geen data)

data_dem_22 = data_dem[data_dem["Year"]==2022].sort_values(by="Democracy score", ascending=False).reset_index().rename(columns={'Entity':'Country_EN'})
data_dem_21 = data_dem[data_dem["Year"]==2021].sort_values(by="Democracy score", ascending=False).reset_index().rename(columns={'Entity':'Country_EN'})
data_dem_20 = data_dem[data_dem["Year"]==2020].sort_values(by="Democracy score", ascending=False).reset_index().rename(columns={'Entity':'Country_EN'})
data_dem_19 = data_dem[data_dem["Year"]==2019].sort_values(by="Democracy score", ascending=False).reset_index().rename(columns={'Entity':'Country_EN'})



# 2022
score = data_dem_22.iloc[0]["Democracy score"]
rank = 1
ranking = []
for country in data_dem_22.index:
    count = country + 1
    if len(ranking)==0:
        ranking.append(1)
    else: 
        if data_dem_22.iloc[country]["Democracy score"] == data_dem_22.iloc[country-1]["Democracy score"]:
            ranking.append(ranking[-1])
        elif data_dem_22.iloc[country]["Democracy score"] < data_dem_22.iloc[country-1]["Democracy score"]:
            ranking.append(count)
            
data_dem_22["ranking"] = ranking


#2021
score = data_dem_21.iloc[0]["Democracy score"]
rank = 1
ranking = []
for country in data_dem_21.index:
    count = country + 1
    if len(ranking)==0:
        ranking.append(1)
    else: 
        if data_dem_21.iloc[country]["Democracy score"] == data_dem_21.iloc[country-1]["Democracy score"]:
            ranking.append(ranking[-1])
        elif data_dem_21.iloc[country]["Democracy score"] < data_dem_21.iloc[country-1]["Democracy score"]:
            ranking.append(count)
            
data_dem_21["ranking"] = ranking


#2020
score = data_dem_20.iloc[0]["Democracy score"]
rank = 1
ranking = []
for country in data_dem_20.index:
    count = country + 1
    if len(ranking)==0:
        ranking.append(1)
    else: 
        if data_dem_20.iloc[country]["Democracy score"] == data_dem_20.iloc[country-1]["Democracy score"]:
            ranking.append(ranking[-1])
        elif data_dem_20.iloc[country]["Democracy score"] < data_dem_20.iloc[country-1]["Democracy score"]:
            ranking.append(count)
            
data_dem_20["ranking"] = ranking


#2019
score = data_dem_19.iloc[0]["Democracy score"]
rank = 1
ranking = []
for country in data_dem_19.index:
    count = country + 1
    if len(ranking)==0:
        ranking.append(1)
    else: 
        if data_dem_19.iloc[country]["Democracy score"] == data_dem_19.iloc[country-1]["Democracy score"]:
            ranking.append(ranking[-1])
        elif data_dem_19.iloc[country]["Democracy score"] < data_dem_19.iloc[country-1]["Democracy score"]:
            ranking.append(count)
            
data_dem_19["ranking"] = ranking




# combine them all
data_dem_combined = pd.concat([data_dem_23, data_dem_22, data_dem_21, data_dem_20, data_dem_19])

### Data voor visualisatie 4

Conflict data voorbereiden

In [65]:
data_conflict = pd.read_csv('./data/conflict.csv').rename(columns={'country': 'Country_EN'})
display(data_conflict)

,Country_EN,Index Category 2023,Index Ranking 2023,Index Category July 2023,Index Ranking July 2023,Change category since July 2023,Change in ranking since July 2023,Deadliness Ranking 2023,Danger Ranking 2023,Diffusion Ranking 2023,Fragmentation Ranking 2023,Deadliness Ranking July 2023,Danger Ranking July 2023,Diffusion Ranking July 2023,Fragmentation Ranking July 2023,Change in ranking since 2019,Change category since 2019,Total score 2023
0,Myanmar,Extreme,1,Extreme,1.0,Consistently concerning,0.0,3,4,9,1,2.0,4.0,8.0,1.0,17,Worsening,2.109
1,Syria,Extreme,2,Extreme,2.0,Consistently concerning,0.0,10,5,4,2,8.0,5.0,3.0,2.0,-1,Consistently concerning,0.987
2,Palestine,Extreme,3,High,11.0,Worsening,8.0,2,3,1,16,34.0,9.0,1.0,17.0,20,Worsening,2.456
3,Mexico,Extreme,4,Extreme,3.0,Consistently concerning,-1.0,8,1,17,5,5.0,1.0,18.0,6.0,-1,Consistently concerning,1.310
4,Nigeria,Extreme,5,Extreme,5.0,Consistently concerning,0.0,5,7,19,9,3.0,6.0,22.0,9.0,6,Worsening,0.728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,Kiribati,Low/Inactive,163,Low/Inactive,167.0,Consistent,4.0,125,161,54,118,125.0,163.0,59.0,125.0,-30,Consistent,0.000
239,Northern Mariana Islands,Low/Inactive,163,Low/Inactive,167.0,Consistent,4.0,125,161,54,118,125.0,163.0,59.0,125.0,-30,Consistent,0.000
240,Palau,Low/Inactive,163,Low/Inactive,167.0,Consistent,4.0,125,161,54,118,125.0,163.0,59.0,125.0,-30,Consistent,0.000
241,Antarctica,Low/Inactive,163,Low/Inactive,167.0,Consistent,4.0,125,161,54,118,125.0,163.0,59.0,125.0,-30,Consistent,0.000


In [66]:
# join de data

data_joined_2 = pd.merge(data_dem_23, data_conflict, on='Country_EN', how='inner')
display(data_joined_2)

,index,Country_EN,Code,Year,Democracy score,ranking,Index Category 2023,Index Ranking 2023,Index Category July 2023,Index Ranking July 2023,...,Danger Ranking 2023,Diffusion Ranking 2023,Fragmentation Ranking 2023,Deadliness Ranking July 2023,Danger Ranking July 2023,Diffusion Ranking July 2023,Fragmentation Ranking July 2023,Change in ranking since 2019,Change category since 2019,Total score 2023
0,1907,Norway,NOR,2023,9.81,1,Low/Inactive,146,Low/Inactive,156.0,...,139,54,118,125.0,145.0,59.0,125.0,-13,Consistent,0.000
1,1796,New Zealand,NZL,2023,9.61,2,Low/Inactive,163,Low/Inactive,167.0,...,161,54,118,125.0,163.0,59.0,125.0,-30,Consistent,0.000
2,1160,Iceland,ISL,2023,9.45,3,Low/Inactive,146,Low/Inactive,167.0,...,139,54,118,125.0,163.0,59.0,125.0,-13,Consistent,0.000
3,2385,Sweden,SWE,2023,9.39,4,Low/Inactive,140,Low/Inactive,139.0,...,128,54,118,125.0,122.0,59.0,125.0,-7,Consistent,0.001
4,904,Finland,FIN,2023,9.30,5,Low/Inactive,146,Low/Inactive,156.0,...,139,54,118,125.0,145.0,59.0,125.0,-13,Consistent,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,2131,Russia,RUS,2023,2.22,151,Turbulent,36,Turbulent,39.0,...,29,40,33,45.0,34.0,45.0,38.0,19,Worsening,0.076
139,1048,Guinea,GIN,2023,2.21,152,Low/Inactive,65,Low/Inactive,77.0,...,64,54,73,66.0,68.0,59.0,86.0,-1,Consistent,0.012
140,936,Gabon,GAB,2023,2.18,153,Low/Inactive,105,Low/Inactive,118.0,...,103,54,85,125.0,107.0,59.0,96.0,12,Consistent,0.003
141,381,Burundi,BDI,2023,2.13,154,Turbulent,32,Turbulent,35.0,...,34,13,37,46.0,33.0,27.0,45.0,-12,Improving,0.085


### Visualisatie 1

Creeer een scatterplot om de relatie tussen democratie en persvrijheid aan te geven:

We set the repository in a way that you can use [Plotly](https://plotly.com/python/) for interactive visualizations.
For more information, check [this documentation](https://jupyterbook.org/en/stable/interactive/interactive.html).

In [67]:
# creeer een scatterplot om de relatie tussen de democratie en persvrijheid aan te geven per land

fig = px.scatter(data_joined, x='ranking', y='Rank', title = 'Democracy index ranking vs Press freedom index ranking 2023 ', hover_data=['Country_EN'], trendline="ols")

fig.update_layout(
    xaxis_title="Democracy index ranking 2023",
    yaxis_title="Press freedom index ranking 2023"
)

for trace in fig.data:
    if trace.mode == "lines":
        trace.line.color = "black"

fig.show()

## Visualisatie 2

Creeer een line chart van de persvrijheid rankings per jaar. Gebruik ctrl+klik om meerdere landen te selecteren

You can also plot the data using static visualizations, such as the [seaborn](https://seaborn.pydata.org/#) library.

In [68]:
# Persvrijheid scores -  ctrl + klik om meerdere landen te selecteren

# Create a dropdown for country selection
country_dropdown = widgets.SelectMultiple(
    options= df_combined_19['EN_country'].unique(),
    value=[df_combined_19['EN_country'].unique()[0]],
    description='Country',
    disabled=False
)

# Function to update the line chart based on selected countries
def update_chart(selected_countries):
    filtered_df = df_combined_19[df_combined_19['EN_country'].isin(selected_countries)]
    fig = px.line(filtered_df, x='Year (N)', y='Rank N', color='EN_country', 
                  labels={'Democracy_score': 'Democracy Score'}, title='Persvrijheid ranking per land')
    fig.update_layout(
        xaxis_title="Year",
        yaxis_title="Persvrijheid rank"
    )
    fig.show()

# Create an interactive output
output = widgets.interactive_output(update_chart, {'selected_countries': country_dropdown})

# Display the dropdown and the output
display(country_dropdown, output)


SelectMultiple(description='Country', index=(0,), options=('Norway', 'Denmark', 'Sweden', 'Netherlands', 'Finl…

Output()

## Visualisatie 3

Creeer een line chart van de democratie rankings per jaar. Gebruik ctrl+klik om meerdere landen te selecteren.

In [69]:
# democratie scores - ctrl + klik om meerdere landen te selecteren

# Create a dropdown for country selection
country_dropdown = widgets.SelectMultiple(
    options= data_dem_combined['Country_EN'].unique(),
    value=[data_dem_combined['Country_EN'].unique()[0]],
    description='Country',
    disabled=False
)

# Function to update the line chart based on selected countries
def update_chart(selected_countries):
    filtered_df = data_dem_combined[data_dem_combined['Country_EN'].isin(selected_countries)]
    fig = px.line(filtered_df, x='Year', y='ranking', color='Country_EN', 
                  labels={'Democracy_score': 'Democracy Score'}, title='Democratie ranking per land')
    fig.update_layout(
        xaxis_title="Year",
        yaxis_title="Democratie rank"
    )
    fig.show()

# Create an interactive output
output = widgets.interactive_output(update_chart, {'selected_countries': country_dropdown})

# Display the dropdown and the output
display(country_dropdown, output)

SelectMultiple(description='Country', index=(0,), options=('Norway', 'New Zealand', 'Iceland', 'Sweden', 'Finl…

Output()

## Visualisatie 4

In [70]:
# creeer een scatterplot om de relatie tussen de democratie en conflict aan te geven per land

fig = px.scatter(data_joined_2, x='ranking', y='Index Ranking 2023', title = 'Democracy index ranking vs Conflict index ranking 2023', hover_data=['Country_EN'], trendline="ols")

fig.update_layout(
    xaxis_title="Democracy index ranking 2023",
    yaxis_title="Conflict index ranking 2023"
)

for trace in fig.data:
    if trace.mode == "lines":
        trace.line.color = "black"


fig.show()

## Ter verduidelijking:


-----
- 'Conflict index':

Een hogere ranking (bijv. 1) geeft aan dat er in een land veel/grote conflicten zijn
Een lagere ranking (bijv. 150) geeft aan dat er in een land weinig/kleine conflicten zijn

--------
- 'Democratie index':

Een hogere ranking (bijv. 1) geeft aan dat er in een land een heersende democratie is
Een lagere ranking (bijv. 150) geeft aan dat er in een land geen democratie is

---------
- 'Persvrijheid  index':

Een hogere ranking (bijv. 1) geeft aan dat de pers in een land veel vrijheid heeft
Een lagere ranking (bijv. 150) geeft aan dat de pers in een land weinig vrijheid heeft.

## Reflectie

Tekst

## Work distribution

Tekst

## Referenties

Tekst